In [2]:
%reset -f
from codice import mymodel
from codice.mymodel import modello_albero
import pyomo.environ as pyo
import numpy as np
import pandas as pd
from scipy.io.arff import loadarff 
from sklearn.model_selection import train_test_split
from codice.mytree import Tree
import importlib
from codice import mynode 
importlib.reload(mymodel)
importlib.reload(mynode)

from sklearn.preprocessing import StandardScaler, MinMaxScaler


SEED = 1

### PREPROCESS DATASET ###
raw_data = loadarff('../dataset/Rice_Cammeo_Osmancik.arff')
df_data = pd.DataFrame(raw_data[0]).to_numpy()
scaler = MinMaxScaler(feature_range = (0, 1))

x_data_raw = scaler.fit_transform(df_data[:, :-1])
#x_data_raw = df_data[:, :-1]
# Cammeo = 0, Osmancik = 1
y_data_raw = [0 if x.decode("utf-8") == "Cammeo" else 1 for x in df_data[:,-1]]

X_train, X_test, y_train, y_test = train_test_split(x_data_raw, y_data_raw, test_size=0.95, stratify=y_data_raw, 
                                                        random_state=SEED) 

num_features = len(X_train[0])


In [3]:
### BUILD PARAMETERS ###
alpha = 0.1 # complexity
beta = 5 # n min of elements in a leaf
delta = 2 # depth

xtrain2 = X_train.copy() 

# compute epsilon value (uj)
u = [None for _ in range(num_features)]
for j in range(num_features):
    xtrain2[:,j].sort()
    for i in range(len(xtrain2)-1):
        diff = abs(xtrain2[i][j] - xtrain2[i+1][j])
        if (u[j] == None or diff < u[j]) and diff != 0:
            u[j] = diff
umax = max(u)
umin = min(u)

## TODO: capire come calcolarlo
lcap = None
c1 = y_train.count(0)
c2 = y_train.count(1)
if c1 > c2:
    lcap = c1/ len(y_train)
else:
    lcap = c2/ len(y_train)

yi_class = [[None for _ in range(len(np.unique(y_train)))] for _ in range(len(X_train))]
for i in range(len(X_train)):
    for col in range(len(np.unique(y_train))):
        yi_class[i][col] = 1 if y_train[i] == col else 0

mvalue = len(X_train)

### BUILD TREE ###
tree = Tree(delta)

In [4]:
import importlib
import time
importlib.reload(mymodel)

tstart = time.time()
modello = mymodel.modello_albero(alpha, beta, delta, u, umax, umin, num_features, X_train, y_train, tree, lcap, yi_class, mvalue)
opt = pyo.SolverFactory('gurobi')
print("START")
results = opt.solve(modello, tee=True) # solves and updates model
pyo.assert_optimal_termination(results)
tend = time.time() - tstart

START
Set parameter Username
Academic license - for non-commercial use only - expires 2025-05-20
Read LP format model from file C:\Users\march\AppData\Local\Temp\tmpy01z9mdy.pyomo.lp
Reading time = 0.02 seconds
x1: 2517 rows, 815 columns, 18369 nonzeros
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 9 6900HS with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 2517 rows, 815 columns and 18369 nonzeros
Model fingerprint: 0xbf4abaec
Variable types: 3 continuous, 812 integer (796 binary)
Coefficient statistics:
  Matrix range     [5e-02, 2e+02]
  Objective range  [1e-01, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Presolve removed 7 rows and 4 columns
Presolve time: 0.02s
Presolved: 2510 rows, 811 columns, 16842 nonzeros
Variable types: 3 continuous, 808 integer (796 binary)
Found heuristic solution: o

In [9]:
print(modello.d[1].value)
print(modello.d[2].value)
print(modello.d[3].value)
print()
print(modello.c[0,4].value)
print(modello.c[0,5].value)
print(modello.c[0,6].value)
print(modello.c[0,7].value)
print()
print(modello.c[1,4].value)
print(modello.c[1,5].value)
print(modello.c[1,6].value)
print(modello.c[1,7].value)

1.0
1.0
1.0

0.0
1.0
0.0
1.0

1.0
0.0
1.0
0.0


In [5]:
amatrix = np.empty((num_features, 2**delta-1))
for riga in range(num_features):
    for col in range(1, 2**delta):
        amatrix[riga, col-1] = modello.a[riga, col].value

cvector = []
for col in range(2**delta, 2**(delta+1)):
    if modello.c[0, col].value == 1:
        cvector.append(0)
    else:
        cvector.append(1)

In [6]:
from codice import mynode
importlib.reload(mynode)

validationtree = mynode.create_complete_tree(1,depth=delta)
mynode.init_bfs(validationtree, avalues=amatrix, bvalues=modello.b, dvalues=modello.d, classnames=cvector, depth=delta)

mynode.print_tree(validationtree)


numcorretti = 0
for i in range(len(y_test)):
    numcorretti += mynode.validate(X_test[i, :], y_test[i], validationtree)

print(numcorretti)
print(len(y_test))
print("Accuracy: ", (numcorretti/len(y_test))*100, "%")

-> Branch1, a:f[0. 0. 1. 0. 0. 0. 0.], b:0.4924134159323148, d:1.0
    -> Leaf2, class:1
    -> Leaf3, class:0
3364
3620
Accuracy:  92.92817679558011 %


In [7]:
import dill
dill.dump_session('notebook_env.db')


1.0